In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
from functools import reduce

import torch
import transformers
import pandas as pd

from tqdm.auto import tqdm

from new_semantic_parsing.utils import get_vocab_top_schema
from new_semantic_parsing.schema_tokenizer import TopSchemaTokenizer

In [3]:
data_path = '../data/top-dataset-semantic-parsing/'

train_data = pd.read_table(data_path + 'train.tsv', names=['text', 'tokens', 'schema'])

In [4]:
schema_vocab = reduce(set.union, map(get_vocab_top_schema, train_data.schema))

In [5]:
text_tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-cased')
schema_tokenizer = TopSchemaTokenizer(schema_vocab, text_tokenizer)

In [6]:
text_ids = [text_tokenizer.encode(text) for text in tqdm(train_data.tokens)]

In [7]:
schema_ids = []
pointer_masks = []

for i, schema in tqdm(enumerate(train_data.schema), total=len(train_data)):

    item = schema_tokenizer.encode_plus(schema, text_ids[i])
    schema_ids.append(item.ids)
    pointer_masks.append(item.pointer_mask)


In [10]:
from new_semantic_parsing.data import PointerDataset

In [17]:
test_data = pd.read_table(data_path + 'test.tsv', names=['text', 'tokens', 'schema'])

test_text_ids = [text_tokenizer.encode(text) for text in tqdm(test_data.tokens)]

In [19]:
len([t for t in test_text_ids if len(t) > 63])

1

In [11]:
dataset = PointerDataset(text_ids, schema_ids, pointer_masks)

In [12]:
dataset.get_max_len()

(63, 98)

In [23]:
dataset.torchify()

In [26]:
torch.save({'train': dataset}, 'dataset.pth')

In [34]:
!mkdir tokenizer

In [35]:
text_tokenizer.save_pretrained('tokenizer')

('tokenizer/vocab.txt',
 'tokenizer/special_tokens_map.json',
 'tokenizer/added_tokens.json')